In [1]:
import torch
import numpy as np
from torch import nn
import torch.utils.data as Data

In [2]:
# load data
train = np.loadtxt("train.txt")
dev = np.loadtxt("dev.txt")
test = np.loadtxt("test.txt")
#vocab size = vocab size:5892

In [3]:
print(train.shape)
print(dev.shape)

(14788, 103)
(5145, 103)


In [4]:
from ignite.metrics import Accuracy
def eval_at_dev(model, dev_iter):
    acc = Accuracy()
    for sent, gd_label in dev_iter:
        pred = model(sent) #get model predict
        acc.update((pred, gd_label))
    acc_rate = acc.compute()
    print("current model over dev set accuracy rate: " + str(acc_rate))
    return acc_rate

In [5]:
#定义数据迭代器
batch_size = 100
train_set = Data.TensorDataset(torch.LongTensor(train[:, 1:103]), torch.LongTensor(train[:, 0]))
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)

dev_batch_size = 343
dev_set = Data.TensorDataset(torch.LongTensor(dev[:, 1:103]), torch.LongTensor(dev[:, 0]))
dev_iter = Data.DataLoader(dev_set, dev_batch_size, shuffle=False)

In [7]:
#定义模型
class LSTM(nn.Module): 
    def __init__(self):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(5895, 100) #5895 个词，每个词的嵌入是100维
        self.lstm = nn.LSTM(input_size=100, hidden_size=100, num_layers=1, batch_first=True)#定义LSTM，LSTM的输入是(batch_size, seq_len, embedding_size)
        self.fc1 = nn.Linear(100, 300)
        self.fc2 = nn.Linear(300, 3)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.lstm(x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        x = x.relu()
        x = self.fc2(x)
        return x

class DAN(nn.Module): 
    def __init__(self):
        super(DAN, self).__init__()
        self.embedding = nn.Embedding(5895, 100) #5895 个词，每个词的嵌入是100维
        self.fc1 = nn.Linear(100, 300)
        self.fc2 = nn.Linear(300, 3)

    def forward(self, x):
        x = self.embedding(x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        x = x.relu()
        x = self.fc2(x)
        return x
#实例化
model = DAN()


In [8]:
#优化

#定义Loss
loss = nn.CrossEntropyLoss()

#定义优化算法
import torch.optim as optim #
optimizer = optim.Adam(model.parameters(), lr=0.001) #使用Adam 优化器

In [9]:
#进行训练
max_epoch = 10
print("before training dev accuracy:")
eval_at_dev(model, dev_iter)
print("let's start training")
for epoch in range(max_epoch):
    for sent, label in train_iter:
        output = model(sent.long())
        L = loss(output, label.long())
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
    eval_at_dev(model, dev_iter)
    print('epoch: %d, loss: %f' % (epoch, L.item()))

before training dev accuracy:
current model over dev set accuracy rate: 0.2639455782312925
let's start training
current model over dev set accuracy rate: 0.6447035957240039
epoch: 0, loss: 0.789752
current model over dev set accuracy rate: 0.7004859086491739
epoch: 1, loss: 0.752326
current model over dev set accuracy rate: 0.7092322643343052
epoch: 2, loss: 0.687499
current model over dev set accuracy rate: 0.7350826044703596
epoch: 3, loss: 0.669652
current model over dev set accuracy rate: 0.7339164237123421
epoch: 4, loss: 0.682741
current model over dev set accuracy rate: 0.7420796890184646
epoch: 5, loss: 0.575167
current model over dev set accuracy rate: 0.726530612244898
epoch: 6, loss: 0.683028
current model over dev set accuracy rate: 0.7484936831875607
epoch: 7, loss: 0.568585
current model over dev set accuracy rate: 0.7216715257531584
epoch: 8, loss: 0.513839
current model over dev set accuracy rate: 0.7523809523809524
epoch: 9, loss: 0.389502
